**<center><font size = "6"> 1. Preprocesamiento de Datos <center>**
***

<h1>Índice<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Limpieza-de-datos" data-toc-modified-id="Limpieza-de-datos-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Limpieza de datos</a></span><ul class="toc-item"><li><span><a href="#Importación-de-los-datasets" data-toc-modified-id="Importación-de-los-datasets-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Importación de los datasets</a></span><ul class="toc-item"><li><span><a href="#AlphaVantage" data-toc-modified-id="AlphaVantage-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>AlphaVantage</a></span></li><li><span><a href="#Twitter" data-toc-modified-id="Twitter-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>Twitter</a></span></li><li><span><a href="#Reddit" data-toc-modified-id="Reddit-1.1.3"><span class="toc-item-num">1.1.3&nbsp;&nbsp;</span>Reddit</a></span></li><li><span><a href="#Reuters" data-toc-modified-id="Reuters-1.1.4"><span class="toc-item-num">1.1.4&nbsp;&nbsp;</span>Reuters</a></span></li><li><span><a href="#Yahoo-Finance" data-toc-modified-id="Yahoo-Finance-1.1.5"><span class="toc-item-num">1.1.5&nbsp;&nbsp;</span>Yahoo Finance</a></span></li><li><span><a href="#Union-de-los-datasets-con-sentimiento" data-toc-modified-id="Union-de-los-datasets-con-sentimiento-1.1.6"><span class="toc-item-num">1.1.6&nbsp;&nbsp;</span>Union de los datasets con sentimiento</a></span></li></ul></li></ul></li><li><span><a href="#Procesamiento-de-texto" data-toc-modified-id="Procesamiento-de-texto-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Procesamiento de texto</a></span><ul class="toc-item"><li><span><a href="#Tokenización,-eliminación-de-stop-words,-lemmatization" data-toc-modified-id="Tokenización,-eliminación-de-stop-words,-lemmatization-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Tokenización, eliminación de stop words, lemmatization</a></span></li><li><span><a href="#Polaridad-de-sentimientos" data-toc-modified-id="Polaridad-de-sentimientos-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Polaridad de sentimientos</a></span><ul class="toc-item"><li><span><a href="#Textblob" data-toc-modified-id="Textblob-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Textblob</a></span></li><li><span><a href="#VADER" data-toc-modified-id="VADER-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>VADER</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>BERT</a></span></li></ul></li></ul></li><li><span><a href="#Almacenamiento-de-features" data-toc-modified-id="Almacenamiento-de-features-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Almacenamiento de features</a></span></li></ul></div>

***

In [1]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob
from transformers import BertTokenizer, TFBertForSequenceClassification
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import nltk
import pandas as pd
import pickle
import re
import tensorflow as tf

# Limpieza de datos

## Importación de los datasets

Se han seleccionado cinco datasets para el desarrollo de este trabajo. Cada dataset proviene de una fuente distinta: Twitter, Reddit, AlphaVantage, Reuters y Yahoo Finanzas.

### AlphaVantage

In [2]:
csv_AV = "./dataset/AlphaVantage_Scraping.csv"
df_AV = pd.read_csv(csv_AV)
df_AV

,Title,URL,Time Published,Authors,Summary,Source,Category Within Source,Source Domain,Overall Sentiment Score,Overall Sentiment Label,Ticker Relevance Score,Ticker Sentiment Score,Ticker Sentiment Label
0,Covid Stops Work at Tesla Plant. But the Stock...,https://www.barrons.com/articles/tesla-tsla-st...,20220316T113506,Al Root,Tesla Plant Halted by Covid. The Stock Rises B...,Barrons,NaN,www.barrons.com,-0.525969,Bearish,0.998682,-0.525969,Bearish
1,Tesla Pauses Bond Sale. Stock Investors Don't ...,https://www.barrons.com/articles/tesla-stock-b...,20220317T142700,Al Root,Tesla Pauses Bond Sale. What It Means for the ...,Barrons,NaN,www.barrons.com,-0.251192,Somewhat-Bearish,0.993371,-0.251192,Somewhat-Bearish
2,Tesla Pauses Bond Sale. Stock Investors Don't ...,https://www.barrons.com/articles/tesla-pauses-...,20220317T115555,Al Root,Tesla has paused a $1 billion asset backed sec...,Barrons,NaN,www.barrons.com,-0.237554,Somewhat-Bearish,0.993371,-0.237554,Somewhat-Bearish
3,Tesla Stock Has Been on a Tear. The Reasons Wh...,https://www.barrons.com/articles/tesla-stock-t...,20220326T005544,Al Root,A Tesla Model 3 Plaid can go from 0 to 60 in j...,Barrons,NaN,www.barrons.com,0.197687,Somewhat-Bullish,0.989159,0.197687,Somewhat-Bullish
4,Tesla Stock Has Been on a Tear. Deliveries Can...,https://www.barrons.com/articles/tesla-stock-t...,20220326T005432,Al Root,Tesla Stock Has Been on a Tear. The Reasons Wh...,Barrons,NaN,www.barrons.com,0.190674,Somewhat-Bullish,0.989159,0.190674,Somewhat-Bullish
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2786,Sila's Gleb Yushin Shows How Immigrants Can Ch...,https://www.forbes.com/sites/stuartanderson/20...,20221212T050627,Stuart Anderson,"Gleb Yushin, an international student to Ameri...",Forbes,Business,www.forbes.com,0.325096,Somewhat-Bullish,0.018935,0.000000,Neutral
2787,Li Auto ( LI ) Q3 2022 Earnings Call Transcript,https://www.fool.com/earnings/call-transcripts...,20221209T200020,Motley Fool Transcribing,LI earnings call for the period ending Septemb...,Motley Fool,NaN,www.fool.com,0.276606,Somewhat-Bullish,0.013557,-0.015396,Neutral
2788,Learning From Successful Investors,https://www.fool.com/investing/2022/12/10/lear...,20221210T140000,Chris Hill,"David Rubenstein, author of ""How to Invest: Ma...",Motley Fool,NaN,www.fool.com,0.257082,Somewhat-Bullish,0.008800,0.126103,Neutral
2789,An Investor's Look at the Energy Picture,https://www.fool.com/investing/2022/12/10/an-i...,20221210T114500,Deidre Woollard,"Plus, we chat about how to get kids into the s...",Motley Fool,NaN,www.fool.com,0.151928,Somewhat-Bullish,0.008647,0.139310,Neutral


In [3]:
# 1. Renombrar la columna 'Time Published' a 'Date'
df_AV.rename(columns={'Time Published': 'Date'}, inplace=True)

# 2. Cambiar el formato de la columna 'Date'  a YYYY-MM-DD
df_AV['Date'] = pd.to_datetime(
    df_AV['Date'], format='%Y%m%dT%H%M%S').dt.strftime('%Y-%m-%d')

# 3. Eliminar registros que no sean del año 2021 o 2022 en la columna Date
df_AV = df_AV[df_AV['Date'].str.contains(r'2021|2022')]

# 4. Concatenar las columnas 'Title' y 'Summary' en una nueva columna 'Body'
df_AV['Body'] = df_AV['Title'] + " " + df_AV['Summary']

# 5. Eliminar las columnas especificadas
columns_to_drop = ['Title', 'URL', 'Summary', 'Source Domain', 'Category Within Source',
                   'Overall Sentiment Score', 'Overall Sentiment Label', 'Ticker Relevance Score', 'Ticker Sentiment Score',
                   'Ticker Sentiment Label']
df_AV.drop(columns=columns_to_drop, inplace=True)

# 6. Renombrar las columnas restantes menos 'Date' y 'Body’ , añadiendo "AV_" al principio
columns_to_rename = [
    col for col in df_AV.columns if col not in ['Date', 'Body']]
df_AV.rename(
    columns={col: f"AV_{col}" for col in columns_to_rename}, inplace=True)

# 7. Agregar una nueva columna 'Origin' con el valor ‘AlphaVantage’ en todas las filas
df_AV['Origin'] = 'AlphaVantage'

In [4]:
df_AV

,Date,AV_Authors,AV_Source,Body,Origin
0,2022-03-16,Al Root,Barrons,Covid Stops Work at Tesla Plant. But the Stock...,AlphaVantage
1,2022-03-17,Al Root,Barrons,Tesla Pauses Bond Sale. Stock Investors Don't ...,AlphaVantage
2,2022-03-17,Al Root,Barrons,Tesla Pauses Bond Sale. Stock Investors Don't ...,AlphaVantage
3,2022-03-26,Al Root,Barrons,Tesla Stock Has Been on a Tear. The Reasons Wh...,AlphaVantage
4,2022-03-26,Al Root,Barrons,Tesla Stock Has Been on a Tear. Deliveries Can...,AlphaVantage
...,...,...,...,...,...
2786,2022-12-12,Stuart Anderson,Forbes,Sila's Gleb Yushin Shows How Immigrants Can Ch...,AlphaVantage
2787,2022-12-09,Motley Fool Transcribing,Motley Fool,Li Auto ( LI ) Q3 2022 Earnings Call Transcr...,AlphaVantage
2788,2022-12-10,Chris Hill,Motley Fool,Learning From Successful Investors David Ruben...,AlphaVantage
2789,2022-12-10,Deidre Woollard,Motley Fool,"An Investor's Look at the Energy Picture Plus,...",AlphaVantage


### Twitter

In [5]:
csv_TW = "./dataset/Elon_Tweets.csv"
df_TW = pd.read_csv(csv_TW)
df_TW

,Unnamed: 0.1,Unnamed: 0,Datetime,Tweet Id,Text,Username,location,reply count,retweet count,like count,language,Twitter Access Point,Follower Count,Friends Count,verified,Date,mentions,sentiment
0,0,0,2022-12-26 00:11:00+00:00,1607167057573085184,@nixcraft lol,elonmusk,NaN,217,53,1439,und,Twitter for iPhone,123153718,162,True,2022-12-26,nixcraft,"['neutral', 0.55792016]"
1,1,1,2022-12-26 00:07:26+00:00,1607166156275879937,@catehall Luck is the greatest superpower,elonmusk,NaN,782,507,7127,en,Twitter for iPhone,123153718,162,True,2022-12-26,catehall,"['positive', 0.8802294]"
2,2,2,2022-12-25 22:56:49+00:00,1607148387799011328,@ashleevance kek,elonmusk,NaN,1550,1229,18401,und,Twitter for iPhone,123153718,162,True,2022-12-25,ashleevance,"['neutral', 0.5784856]"
3,3,3,2022-12-25 20:47:12+00:00,1607115769967693824,@charlieifrah @drmichaellevin Potentially majo...,elonmusk,NaN,290,182,2819,en,Twitter for iPhone,123153718,162,True,2022-12-25,"charlieifrah,drmichaellevin","['negative', 0.6977931]"
4,4,4,2022-12-25 20:39:51+00:00,1607113920275439617,@stats_feed Great work by @SpaceX Falcon Team!,elonmusk,NaN,604,865,23666,en,Twitter for iPhone,123153718,162,True,2022-12-25,"stats_feed,SpaceX","['positive', 0.9687216]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19185,19185,19185,2011-12-03 08:22:07+00:00,142881284019060736,That was a total non sequitur btw,elonmusk,NaN,42,15,142,en,Twitter for iPhone,123154426,162,True,2011-12-03,_,"['negative', 0.5067324]"
19186,19186,19186,2011-12-03 08:20:28+00:00,142880871391838208,"Great Voltaire quote, arguably better than Twa...",elonmusk,NaN,36,33,85,en,Twitter for iPhone,123154426,162,True,2011-12-03,_,"['positive', 0.55860823]"
19187,19187,19187,2011-12-01 10:29:04+00:00,142188458125963264,I made the volume on the Model S http://t.co/w...,elonmusk,NaN,29,11,79,en,Twitter Web Client,123154426,162,True,2011-12-01,_,"['neutral', 0.6384011]"
19188,19188,19188,2011-12-01 09:55:11+00:00,142179928203460608,Went to Iceland on Sat to ride bumper cars on ...,elonmusk,NaN,41,23,192,en,Twitter Web Client,123154426,162,True,2011-12-01,_,"['positive', 0.9733198]"


In [6]:
df_TW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19190 entries, 0 to 19189
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0.1          19190 non-null  int64  
 1   Unnamed: 0            19190 non-null  int64  
 2   Datetime              19190 non-null  object 
 3   Tweet Id              19190 non-null  int64  
 4   Text                  19190 non-null  object 
 5   Username              19190 non-null  object 
 6   location              0 non-null      float64
 7   reply count           19190 non-null  int64  
 8   retweet count         19190 non-null  int64  
 9   like count            19190 non-null  int64  
 10  language              19190 non-null  object 
 11  Twitter Access Point  19190 non-null  object 
 12  Follower Count        19190 non-null  int64  
 13   Friends Count        19190 non-null  int64  
 14  verified              19190 non-null  bool   
 15  Date               

In [7]:
# 1. Eliminar registros que no sean del año 2021 o 2022 en la columna Date
df_TW['Date'] = pd.to_datetime(df_TW['Date'])
df_TW = df_TW[df_TW['Date'].dt.year.isin([2021, 2022])].copy()

# 2. Copiar el contenido de la columna 'Text' a una nueva columna 'Body' sin emojis ni menciones
df_TW['Body'] = df_TW['Text'].str.replace(
    r'(@\w+)|([^a-zA-Z0-9\s])', '', regex=True)

# 3. Eliminar aquellos registros que no tengan al menos una letra en la columna ‘Body’
df_TW = df_TW[df_TW['Body'].str.contains(r'[a-zA-Z]')]

# 4. Eliminar las columnas especificadas
columns_to_drop = ['Unnamed: 0.1', 'Unnamed: 0', 'Datetime', 'Tweet Id', 'Text', 'Username',
                   'location', 'language', 'reply count', 'Twitter Access Point', 'Follower Count',
                   ' Friends Count', 'verified', 'mentions', 'sentiment']
df_TW.drop(columns=columns_to_drop, inplace=True)

# 5. Renombrar las columnas restantes menos 'Date' y 'Body’ , añadiendo "TW_" al principio
columns_to_rename = [
    col for col in df_TW.columns if col not in ['Date', 'Body']]
df_TW.rename(
    columns={col: f"TW_{col}" for col in columns_to_rename}, inplace=True)

# 6. Agregar una nueva columna 'Origin' con el valor ‘Twitter’ en todas las filas
df_TW['Origin'] = 'Twitter'

In [8]:
df_TW

,TW_retweet count,TW_like count,Date,Body,Origin
0,53,1439,2022-12-26,lol,Twitter
1,507,7127,2022-12-26,Luck is the greatest superpower,Twitter
2,1229,18401,2022-12-25,kek,Twitter
3,182,2819,2022-12-25,Potentially major problem,Twitter
4,865,23666,2022-12-25,Great work by Falcon Team,Twitter
...,...,...,...,...,...
7884,634,12461,2021-01-02,Tesla is responsible for 23 of all the person...,Twitter
7885,5775,103699,2021-01-02,So proud of the Tesla team for achieving this ...,Twitter
7886,2191,49681,2021-01-02,Um we have giant fusion reactor in the sky th...,Twitter
7887,865,19570,2021-01-02,Tesla Full SelfDriving will work at a safety ...,Twitter


### Reddit

In [9]:
csv_RDT = "./dataset/Reddit_Scraping.csv"
df_RDT = pd.read_csv(csv_RDT)
df_RDT

,title,score,id,subreddit,category,url,num_comments,body,created
0,Tesla (TSLA) is delivering its first ever all-...,1690,z9r5ww,StockMarket,Discussion,https://i.redd.it/oxpjohy05b3a1.jpg,522,NaN,1.669910e+09
1,"Elon Musk has recently tweeted ""I will not let...",1403,y8crbq,StockMarket,Discussion,https://i.redd.it/eocf89agjtu91.png,351,NaN,1.666210e+09
2,TSLA - time to VOTE!,2533,qoojdr,StockMarket,News,https://i.redd.it/rkxwbw1wd6y71.png,343,NaN,1.636292e+09
3,Mercedes unveils new electric concept cars wit...,465,16b72u1,StockMarket,News,https://finance.yahoo.com/news/mercedes-unveil...,231,NaN,1.693964e+09
4,Do you think tesla (TSLA) might rally heading ...,785,1129n4k,StockMarket,Discussion,https://i.redd.it/7qosqv0ml6ia1.png,284,NaN,1.676392e+09
...,...,...,...,...,...,...,...,...,...
9995,Ford’s MachE – a examination of Tesla’s compet...,13,vlyf6r,StockMarket,Discussion,https://www.reddit.com/r/StockMarket/comments/...,6,TL/DR upfront:\n\nUpcoming EV competition is *...,1.656345e+09
9996,Intraday trade recap on TSLA,2,xw82f6,StockMarket,Recap/Watchlist,https://ko-fi.com/post/Intraday-Trade-Recap--T...,0,NaN,1.664969e+09
9997,First time in over a year that $TSLA has had 5...,6,wv2n0z,StockMarket,News,https://i.redd.it/hywgg7wtfbj91.jpg,2,NaN,1.661196e+09
9998,TSLA shining green today while MRNA looking br...,41,rmf5dg,StockMarket,Recap/Watchlist,https://i.imgur.com/OJEaZKO.png,13,NaN,1.640208e+09


In [10]:
df_RDT.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         10000 non-null  object 
 1   score         10000 non-null  int64  
 2   id            10000 non-null  object 
 3   subreddit     10000 non-null  object 
 4   category      9300 non-null   object 
 5   url           10000 non-null  object 
 6   num_comments  10000 non-null  int64  
 7   body          4100 non-null   object 
 8   created       10000 non-null  float64
dtypes: float64(1), int64(2), object(6)
memory usage: 703.2+ KB


In [11]:
# 1. Cambiar el formato de la columna 'Date' de 1.664969e+09 a YYYY-MM-DD
df_RDT['Date'] = pd.to_datetime(
    df_RDT['created'], unit='s').dt.strftime('%Y-%m-%d')

# 2. Eliminar registros que no sean del año 2021 o 2022 en la columna Date
df_RDT = df_RDT[df_RDT['Date'].str.contains(r'2021|2022')].copy()

# 3. Concatenar las columnas 'title' y 'body' en una nueva columna 'Body'
df_RDT['Body'] = df_RDT['title'] + " " + df_RDT['body'].fillna('')

# 4. Eliminar las columnas especificadas
columns_to_drop = ['title', 'id', 'subreddit', 'url', 'body', 'created']
df_RDT.drop(columns=columns_to_drop, inplace=True)

# 5. Renombrar las columnas restantes menos 'Date' y 'Body’ , añadiendo "RDT_" al principio
columns_to_rename = [
    col for col in df_RDT.columns if col not in ['Date', 'Body']]
df_RDT.rename(
    columns={col: f"RDT_{col}" for col in columns_to_rename}, inplace=True)

# 6. Agregar una nueva columna 'Origin' con el valor ‘Reddit’ en todas las filas
df_RDT['Origin'] = 'Reddit'

In [12]:
df_RDT

,RDT_score,RDT_category,RDT_num_comments,Date,Body,Origin
0,1690,Discussion,522,2022-12-01,Tesla (TSLA) is delivering its first ever all-...,Reddit
1,1403,Discussion,351,2022-10-19,"Elon Musk has recently tweeted ""I will not let...",Reddit
2,2533,News,343,2021-11-07,TSLA - time to VOTE!,Reddit
5,589,News,254,2022-12-19,Elizabeth Warren has said: Elon Musk took over...,Reddit
8,365,News,172,2022-12-31,"*TESLA STOCK FELL 65% IN 2022, THE BIGGEST ANN...",Reddit
...,...,...,...,...,...,...
9994,0,Valuation,26,2021-07-25,$TSLA Will be the largest company on earth - T...,Reddit
9995,13,Discussion,6,2022-06-27,Ford’s MachE – a examination of Tesla’s compet...,Reddit
9996,2,Recap/Watchlist,0,2022-10-05,Intraday trade recap on TSLA,Reddit
9997,6,News,2,2022-08-22,First time in over a year that $TSLA has had 5...,Reddit


### Reuters

In [13]:
csv_RTS = "./dataset/Reuters_Scraping.csv"
df_RTS = pd.read_csv(csv_RTS)
df_RTS

,Section,Title,Date
0,World at Work,UAW union embraces social media as labor talks...,"September 9, 2023"
1,Legal,Elon Musk's X Corp sues California to undo con...,"September 9, 2023"
2,Technology,Congress to hold new AI hearings as it works t...,"September 8, 2023"
3,Business,"China car sales return to growth in August, Te...","September 8, 2023"
4,Insight,Why Native American tribes struggle to tap bil...,"September 8, 2023"
...,...,...,...
3964,Analysis,"U.S. stocks surge in recovery; dollar, bond yi...","May 14, 2021"
3965,Business,Wall St extends recovery at end of volatile week,"May 14, 2021"
3966,Business,"Dollar lower after weak data, as inflation fea...","May 14, 2021"
3967,World,"Take Five: Retail, PMIs and a timely tantrum a...","May 14, 2021"


In [14]:
df_RTS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3969 entries, 0 to 3968
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Section  3969 non-null   object
 1   Title    3969 non-null   object
 2   Date     3969 non-null   object
dtypes: object(3)
memory usage: 93.1+ KB


In [15]:
# 1. Cambiar el formato de la columna 'Date' a YYYY-MM-DD
df_RTS['Date'] = pd.to_datetime(df_RTS['Date'])

# 2. Eliminar registros que no sean del año 2021 o 2022
df_RTS = df_RTS[df_RTS['Date'].dt.year.isin([2021, 2022])].copy()

# 3. Renombrar la columna 'Title' a 'Body'
df_RTS.rename(columns={'Title': 'Body'}, inplace=True)

# 4. Renombrar las columnas restantes menos 'Date' y 'Body’ , añadiendo "RTS_" al principio
columns_to_rename = [
    col for col in df_RTS.columns if col not in ['Date', 'Body']]
df_RTS.rename(
    columns={col: f"RTS_{col}" for col in columns_to_rename}, inplace=True)

# 5. Agregar una nueva columna 'Origin' con el valor ‘Reuters’ en todas las filas
df_RTS['Origin'] = 'Reuters'

In [16]:
df_RTS

,RTS_Section,Body,Date,Origin
1207,Markets,Wall Street ends 2022 with biggest annual drop...,2022-12-30,Reuters
1208,Macro Matters,"Inflation, recession and earnings among factor...",2022-12-30,Reuters
1209,Markets,"How 2022 shocked, rocked and rolled global mar...",2022-12-30,Reuters
1210,Disrupted,"Baidu, Pony.ai start driverless robotaxi tests...",2022-12-30,Reuters
1211,Markets,"Wall St ends firmer, growth stocks lead in thi...",2022-12-30,Reuters
...,...,...,...,...
3964,Analysis,"U.S. stocks surge in recovery; dollar, bond yi...",2021-05-14,Reuters
3965,Business,Wall St extends recovery at end of volatile week,2021-05-14,Reuters
3966,Business,"Dollar lower after weak data, as inflation fea...",2021-05-14,Reuters
3967,World,"Take Five: Retail, PMIs and a timely tantrum a...",2021-05-14,Reuters


### Yahoo Finance

In [17]:
csv_TSLA = "./dataset/TSLA.csv"
df_TSLA = pd.read_csv(csv_TSLA)
df_TSLA

,Date,Open,High,Low,Close,Adj Close,Volume
0,2021-01-04,239.820007,248.163330,239.063339,243.256668,243.256668,145914600
1,2021-01-05,241.220001,246.946671,239.733337,245.036667,245.036667,96735600
2,2021-01-06,252.830002,258.000000,249.699997,251.993332,251.993332,134100000
3,2021-01-07,259.209991,272.329987,258.399994,272.013336,272.013336,154496700
4,2021-01-08,285.333344,294.829987,279.463318,293.339996,293.339996,225166500
...,...,...,...,...,...,...,...
498,2022-12-23,126.370003,128.619995,121.019997,123.150002,123.150002,166989700
499,2022-12-27,117.500000,119.669998,108.760002,109.099998,109.099998,208643400
500,2022-12-28,110.349998,116.269997,108.239998,112.709999,112.709999,221070500
501,2022-12-29,120.389999,123.570000,117.500000,121.820000,121.820000,221923300


In [18]:
df_TSLA['Date'] = pd.to_datetime(df_TSLA['Date'])
df_TSLA = df_TSLA.sort_values('Date')

In [19]:
df_TSLA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 503 entries, 0 to 502
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       503 non-null    datetime64[ns]
 1   Open       503 non-null    float64       
 2   High       503 non-null    float64       
 3   Low        503 non-null    float64       
 4   Close      503 non-null    float64       
 5   Adj Close  503 non-null    float64       
 6   Volume     503 non-null    int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 31.4 KB


### Union de los datasets con sentimiento

In [20]:
# Concatenar los DataFrames uno debajo del otro
merged_df = pd.concat([df_AV, df_TW, df_RDT, df_RTS], ignore_index=True)

# Agrupar por columnas con el mismo nombre y mergear su contenido
df = merged_df.groupby(merged_df.columns, axis=1).sum().reset_index(drop=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18174 entries, 0 to 18173
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AV_Authors        18174 non-null  object 
 1   AV_Source         18174 non-null  object 
 2   Body              18174 non-null  object 
 3   Date              18174 non-null  object 
 4   Origin            18174 non-null  object 
 5   RDT_category      18174 non-null  object 
 6   RDT_num_comments  18174 non-null  float64
 7   RDT_score         18174 non-null  float64
 8   RTS_Section       18174 non-null  object 
 9   TW_like count     18174 non-null  float64
 10  TW_retweet count  18174 non-null  float64
dtypes: float64(4), object(7)
memory usage: 1.5+ MB


In [22]:
df = df.convert_dtypes()

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18174 entries, 0 to 18173
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   AV_Authors        18174 non-null  object
 1   AV_Source         18174 non-null  object
 2   Body              18174 non-null  string
 3   Date              18174 non-null  object
 4   Origin            18174 non-null  string
 5   RDT_category      18174 non-null  object
 6   RDT_num_comments  18174 non-null  Int64 
 7   RDT_score         18174 non-null  Int64 
 8   RTS_Section       18174 non-null  object
 9   TW_like count     18174 non-null  Int64 
 10  TW_retweet count  18174 non-null  Int64 
dtypes: Int64(4), object(5), string(2)
memory usage: 1.6+ MB


In [24]:
df["Date"] = pd.to_datetime(df["Date"])

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18174 entries, 0 to 18173
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   AV_Authors        18174 non-null  object        
 1   AV_Source         18174 non-null  object        
 2   Body              18174 non-null  string        
 3   Date              18174 non-null  datetime64[ns]
 4   Origin            18174 non-null  string        
 5   RDT_category      18174 non-null  object        
 6   RDT_num_comments  18174 non-null  Int64         
 7   RDT_score         18174 non-null  Int64         
 8   RTS_Section       18174 non-null  object        
 9   TW_like count     18174 non-null  Int64         
 10  TW_retweet count  18174 non-null  Int64         
dtypes: Int64(4), datetime64[ns](1), object(4), string(2)
memory usage: 1.6+ MB


# Procesamiento de texto

## Tokenización, eliminación de stop words, lemmatization


In [26]:
# Descargar las stopwords y el lematizador de WordNet
nltk.download('stopwords')
nltk.download('wordnet')


def preprocess_text(text):
    # Paso 1: Convertir a minúsculas
    text = text.lower()

    # Paso 2: Eliminar stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(filtered_words)

    # Paso 3: Lematización
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    text = ' '.join(lemmatized_words)

    # Paso 4: Eliminar espacios en blanco adicionales
    text = ' '.join(text.split())

    # Paso 5: Eliminar URLs
    text = re.sub(r'http\S+', '', text)

    # Paso 6: Eliminar texto repetido
    text = re.sub(r'\b(\w+\b)\s+\1', r'\1', text)

    return text


# Aplicar la función a la columna 'Body' y crear una nueva columna 'Processed_Body'
df['Processed_Body'] = df['Body'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Agus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Agus\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Polaridad de sentimientos

### Textblob


In [27]:
# Función para calcular la polaridad usando TextBlob
def calculate_polarity(text):
    analysis = TextBlob(str(text))
    return analysis.sentiment.polarity


# Aplicar la función a la columna 'Body' y crear una nueva columna 'TextBlob_Polarity'
df['TextBlob_Polarity'] = df['Processed_Body'].apply(calculate_polarity)

### VADER

In [28]:
# Función para calcular la polaridad usando VADER
def calculate_vader_polarity(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(str(text))
    return sentiment['compound']


# Aplicar la función a la columna 'Body' y crear una nueva columna 'VADER_Polarity'
df['VADER_Polarity'] = df['Processed_Body'].apply(calculate_vader_polarity)

### BERT

In [29]:
# Crear un tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Calcular el número máximo de tokens en la columna 'Body'
max_tokens = df['Processed_Body'].apply(lambda x: len(
    tokenizer.encode(x, add_special_tokens=True))).max()

print("Número máximo de tokens en la columna 'Body':", max_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (1016 > 512). Running this sequence through the model will result in indexing errors


Número máximo de tokens en la columna 'Body': 1016


In [30]:
# Cargar el modelo BERT y el tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)

# Función para calcular la polaridad usando BERT
def calculate_bert_polarity(text):
    # Tokenizar el texto y obtener los IDs de los tokens
    tokens = tokenizer.tokenize(text)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Dividir los tokens en fragmentos de longitud máxima admitida (por ejemplo, 512 tokens)
    max_seq_length = 512
    token_chunks = [token_ids[i:i + max_seq_length]
                    for i in range(0, len(token_ids), max_seq_length)]

    # Calcular la polaridad para cada fragmento y agregarlos a la lista de polaridades
    polarities = []
    for chunk in token_chunks:
        inputs = tokenizer.encode_plus(
            chunk, return_tensors='tf', truncation=True)
        outputs = model(inputs)
        logits = outputs.logits
        probabilities = tf.nn.softmax(logits, axis=1)
        # Suponiendo etiquetas 0 y 1
        polarity = probabilities[0][1].numpy() - probabilities[0][0].numpy()
        polarities.append(polarity)

    # Calcular el promedio de las polaridades de los fragmentos si la lista no está vacía
    if polarities:
        average_polarity = sum(polarities) / len(polarities)
    else:
        average_polarity = 0.0  # Si no se encontraron fragmentos, asignar polaridad cero

    return average_polarity


# Aplicar la función a la columna 'Processed_Body' y crear una nueva columna 'BERT_Polarity'
df['BERT_Polarity'] = df['Processed_Body'].apply(calculate_bert_polarity)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Almacenamiento de features

In [31]:
pickle.dump(df, open("saved_feats/df", "wb"))
pickle.dump(df_TSLA, open("saved_feats/df_TSLA", "wb"))